<a href="https://colab.research.google.com/github/ankush-003/NoSQLi-Detector/blob/main/NoSQLi_Generator_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U accelerate
!pip install -U transformers
!pip install --upgrade huggingface_hub

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from google.colab import files

uploaded = files.upload()
fnames = []
for fn in uploaded.keys():
  fnames.append(fn)
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

In [ ]:
print(f"/content/{fnames[0]}")

/content/nosql_injection_dataset (1).json


In [ ]:
import torch
import accelerate
from transformers import DistilBertForMaskedLM, DistilBertTokenizer, DistilBertConfig
from transformers import Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import json

# Define the dataset class
class NoSQLInjectionDataset(Dataset):
    def __init__(self, dataset, tokenizer):
        self.dataset = dataset
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        template = self.dataset[idx]["template"]
        payload = self.dataset[idx]["payload"]

        # Tokenize template and payload
        tokenized_inputs = self.tokenizer.encode_plus(
            template,
            payload,
            truncation=True,
            padding="max_length",
            max_length=512,  # Adjust as needed
            return_tensors="pt"
        )

        input_ids = tokenized_inputs["input_ids"].squeeze()
        attention_mask = tokenized_inputs["attention_mask"].squeeze()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids  # Use input_ids as labels for masked language modeling
        }

# Load the generated dataset
with open(f"/content/{fnames[0]}", "r") as f:
    dataset = json.load(f)

# Load the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
config = DistilBertConfig.from_pretrained(model_name)
model = DistilBertForMaskedLM(config)

# Convert the dataset into a PyTorch Dataset
train_dataset = NoSQLInjectionDataset(dataset, tokenizer)

# Set up the training arguments and trainer
training_args = TrainingArguments(
    output_dir="./training_output",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    save_steps=500,
    logging_steps=1000,
    logging_dir="./logs",
    overwrite_output_dir=True,
    learning_rate=5e-5,  # Adjust as needed
    warmup_steps=100,  # Adjust as needed
    weight_decay=0.01  # Adjust as needed
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)

# Fine-tune the model
trainer.train()

# Save the fine-tuned model
model.save_pretrained("fine-tuned-distilbert")

# Push the fine-tuned model to the Hugging Face Model Hub
model.push_to_hub("fine-tuned-distilbert-nosql-injection")

print("Fine-tuned model saved and pushed to the Hugging Face Model Hub.")

NameError: ignored

In [ ]:
from transformers import pipeline

# Load the fine-tuned model
model_name = "your-username/fine-tuned-distilbert-nosql-injection"
fill_mask = pipeline("fill-mask", model=model_name, tokenizer=model_name)

# Provide a masked input and generate predictions
masked_input = "{'username': '[MASK]', 'password': 'password'}"
predictions = fill_mask(masked_input)

# Print the predicted outputs
for prediction in predictions:
    print(f"Prediction: {prediction['sequence']} (Score: {prediction['score']:.4f})")